In [1]:
import pandas as pd
import numpy as np


In [60]:
#read csv files in data
train = pd.read_csv('data/train.csv')
test= pd.read_csv('data/test.csv')

In [61]:
#only show train that has dtype=object
train.select_dtypes(include=['object']).columns

Index(['product_type', 'sub_area', 'culture_objects_top_25',
       'thermal_power_plant_raion', 'incineration_raion',
       'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion',
       'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion',
       'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology'],
      dtype='object')

In [62]:
#convert data types where coluhmns have values yes and no to 1 and 0
train = train.replace({'yes': 1, 'no': 0})
test = test.replace({'yes': 1, 'no': 0})

In [63]:
#make surre all yes and no are covnerted
train.select_dtypes(include=['object']).columns

Index(['product_type', 'sub_area', 'ecology'], dtype='object')

In [64]:
#  Import the necessary libraries
from sklearn.preprocessing import LabelEncoder

In [65]:

#Create an instance of LabelEncoder
lbl = LabelEncoder()

In [66]:

#Fit the label encoder on the combined data of train and test datasets
lbl.fit(list(train['ecology'].values) + list(test['ecology'].values))

LabelEncoder()

In [67]:

#Convert the 'ecology' column in the train dataset
train['ecology'] = lbl.transform(list(train['ecology'].values))

In [68]:

#Convert the 'ecology' column in the test dataset
test['ecology'] = lbl.transform(list(test['ecology'].values))

In [69]:
#do label encoding for prodcut type as well
lbl.fit(list(train['product_type'].values) + list(test['product_type'].values))
train['product_type'] = lbl.transform(list(train['product_type'].values))
test['product_type'] = lbl.transform(list(test['product_type'].values))

In [70]:
# Get the top 10 sub areas
top_10_sub_areas = train['sub_area'].value_counts().nlargest(10).index.tolist()

# Replace all other sub areas with "Others"
train.loc[~train['sub_area'].isin(top_10_sub_areas), 'sub_area'] = 'Others'
test.loc[~test['sub_area'].isin(top_10_sub_areas), 'sub_area'] = 'Others'

In [71]:
#show unique values in train and test
train['sub_area'].unique()  
test['sub_area'].unique()

array(['Others', "Chertanovo Central'noe", 'Tverskoe', 'Alekseevskoe',
       'Losinoostrovskoe', 'Shhukino', 'Caricyno', 'Prospekt Vernadskogo',
       'Poselenie Voskresenskoe', 'Poselenie Mosrentgen',
       'Poselenie Shhapovskoe'], dtype=object)

In [72]:
train = pd.get_dummies(train, columns=['sub_area'])
test = pd.get_dummies(test, columns=['sub_area'])

In [13]:
train.drop(columns=['sub_area'], inplace=True)
test.drop(columns=['sub_area'], inplace=True)

In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [75]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [76]:
X = train.drop(columns=['price_doc'])
y = train[['price_doc']]

In [77]:
#save row id in a dataframe
row_id = test['row ID']
#drop it from test now
test = test.drop(columns=['row ID'])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [19]:
from sklearn.feature_selection import SelectKBest, f_regression

#Select top 30 features
selector = SelectKBest(f_regression, k=30)
X_new = selector.fit_transform(X, y)
X_new = pd.DataFrame(X_new)

C:\Users\hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
X_new = scaler.fit_transform(X_new)


In [21]:

#Split the data
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.3)

#Train the model
reg2 = LinearRegression().fit(X_train, y_train)

#Print R2 score and coefficients
print(reg2.score(X_train, y_train))
print(reg2.coef_)
print(reg2.intercept_)

#Predict on the test set
y_pred_test = reg2.predict(X_test)

#Calculate and print RMSE
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
print("Root Mean squared error: %.2f" % rmse)

0.6058679723951985
[[11862702.30905803  9062191.01037043   980578.71157445  3114945.26560373
   6367646.49059357   504604.46030695  1513017.92620245   110494.693702
   3339198.72127779  2829012.11712643  -557834.24889954  4227228.7503756
   5768058.23613711  2600383.32405641   896936.96744038  9880639.8874198
   9514462.02799702  5636718.08470158  -343424.46457552   242627.88397651
   1519958.20436663   837666.78981777  -455019.79739402  5676763.66060707
   1519797.68049536  6416199.04141568   818378.14174398   961669.32489456
   2146403.77013997  -741443.03743489]]
[5735507.96459618]
Root Mean squared error: 13694714.68


In [22]:
reg2 = LinearRegression().fit(X_new, y)

In [27]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
testsclae = scaler.fit_transform(test)


In [28]:
Xtest = selector.transform(testsclae)

C:\Users\hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SelectKBest was fitted with feature names
  warnings.warn(


In [78]:
import statsmodels.api as sm

# Add a constant column to the feature matrix
XSM = sm.add_constant(train.drop(columns=['price_doc']))

# Fit the ordinary least squares (OLS) model
model = sm.OLS(train['price_doc'], XSM).fit()

# Get the p-values of all columns
p_values = model.pvalues

# Sort the p-values in ascending order
sorted_p_values = p_values.sort_values()

# Show only the bottom 35 columns with the lowest p-values
bottom_35_p_values = sorted_p_values[:35]

# Print the bottom 35 p-values
# print(bottom_35_p_values)


In [83]:
# Use the bottom 35 features above to train the model
if 'const' in bottom_35_p_values:
    bottom_35_p_values = bottom_35_p_values.drop('const')


In [84]:
bottom_35_p_values

sub_area_Tverskoe                        0.000000e+00
floor                                    1.353870e-57
school_education_centers_top_20_raion    2.602256e-54
full_sq                                  4.296093e-51
sub_area_Others                          2.837175e-45
sub_area_Prospekt Vernadskogo            1.777046e-36
leisure_count_500                        2.279875e-36
industrial_km                            1.777338e-35
product_type                             6.788024e-29
culture_objects_top_25_raion             1.160549e-28
leisure_count_1000                       3.182589e-28
green_part_1000                          1.893579e-25
build_count_monolith                     2.579734e-20
cafe_count_500_price_high                2.339915e-19
mosque_count_500                         3.912821e-18
sub_area_Losinoostrovskoe                1.563551e-17
sport_count_2000                         4.135465e-17
culture_objects_top_25                   6.374648e-17
cafe_count_1000_price_high  

In [85]:
# Select the bottom 35 variables from the dataset
Xp = train[list(bottom_35_p_values.index)]
# Do the same for the test dataset as well
testp = test[list(bottom_35_p_values.index)]

In [86]:
from sklearn.preprocessing import MinMaxScaler

#Create a scaler object
scaler = MinMaxScaler()

#Fit the scaler and transform X_new
testp = scaler.fit_transform(testp)
Xp = scaler.fit_transform(Xp)

In [87]:
#Xp and testp are the new datasets with only the bottom 35 features
poly = PolynomialFeatures(2, include_bias=False)
X2 = poly.fit_transform(Xp)
X2_test=poly.transform(testp)
print(X2.shape)
# print(test.shape)
print(poly.get_feature_names_out())
reg = LinearRegression().fit(X2,y)
print(reg.score(X2, y))
print(reg.coef_)
print(reg.intercept_)

(181507, 629)
['x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x6' 'x7' 'x8' 'x9' 'x10' 'x11' 'x12' 'x13'
 'x14' 'x15' 'x16' 'x17' 'x18' 'x19' 'x20' 'x21' 'x22' 'x23' 'x24' 'x25'
 'x26' 'x27' 'x28' 'x29' 'x30' 'x31' 'x32' 'x33' 'x0^2' 'x0 x1' 'x0 x2'
 'x0 x3' 'x0 x4' 'x0 x5' 'x0 x6' 'x0 x7' 'x0 x8' 'x0 x9' 'x0 x10' 'x0 x11'
 'x0 x12' 'x0 x13' 'x0 x14' 'x0 x15' 'x0 x16' 'x0 x17' 'x0 x18' 'x0 x19'
 'x0 x20' 'x0 x21' 'x0 x22' 'x0 x23' 'x0 x24' 'x0 x25' 'x0 x26' 'x0 x27'
 'x0 x28' 'x0 x29' 'x0 x30' 'x0 x31' 'x0 x32' 'x0 x33' 'x1^2' 'x1 x2'
 'x1 x3' 'x1 x4' 'x1 x5' 'x1 x6' 'x1 x7' 'x1 x8' 'x1 x9' 'x1 x10' 'x1 x11'
 'x1 x12' 'x1 x13' 'x1 x14' 'x1 x15' 'x1 x16' 'x1 x17' 'x1 x18' 'x1 x19'
 'x1 x20' 'x1 x21' 'x1 x22' 'x1 x23' 'x1 x24' 'x1 x25' 'x1 x26' 'x1 x27'
 'x1 x28' 'x1 x29' 'x1 x30' 'x1 x31' 'x1 x32' 'x1 x33' 'x2^2' 'x2 x3'
 'x2 x4' 'x2 x5' 'x2 x6' 'x2 x7' 'x2 x8' 'x2 x9' 'x2 x10' 'x2 x11'
 'x2 x12' 'x2 x13' 'x2 x14' 'x2 x15' 'x2 x16' 'x2 x17' 'x2 x18' 'x2 x19'
 'x2 x20' 'x2 x21' 'x2 x22' 'x2 x23' 'x2 x24

In [29]:
poly = PolynomialFeatures(2, include_bias=False)
X2 = poly.fit_transform(X_new)
X2_test=poly.transform(Xtest)
print(X2.shape)
print(test.shape)
print(poly.get_feature_names_out())
reg = LinearRegression().fit(X2,y)
print(reg.score(X2, y))
print(reg.coef_)
print(reg.intercept_)

(181507, 495)
(77789, 270)
['x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x6' 'x7' 'x8' 'x9' 'x10' 'x11' 'x12' 'x13'
 'x14' 'x15' 'x16' 'x17' 'x18' 'x19' 'x20' 'x21' 'x22' 'x23' 'x24' 'x25'
 'x26' 'x27' 'x28' 'x29' 'x0^2' 'x0 x1' 'x0 x2' 'x0 x3' 'x0 x4' 'x0 x5'
 'x0 x6' 'x0 x7' 'x0 x8' 'x0 x9' 'x0 x10' 'x0 x11' 'x0 x12' 'x0 x13'
 'x0 x14' 'x0 x15' 'x0 x16' 'x0 x17' 'x0 x18' 'x0 x19' 'x0 x20' 'x0 x21'
 'x0 x22' 'x0 x23' 'x0 x24' 'x0 x25' 'x0 x26' 'x0 x27' 'x0 x28' 'x0 x29'
 'x1^2' 'x1 x2' 'x1 x3' 'x1 x4' 'x1 x5' 'x1 x6' 'x1 x7' 'x1 x8' 'x1 x9'
 'x1 x10' 'x1 x11' 'x1 x12' 'x1 x13' 'x1 x14' 'x1 x15' 'x1 x16' 'x1 x17'
 'x1 x18' 'x1 x19' 'x1 x20' 'x1 x21' 'x1 x22' 'x1 x23' 'x1 x24' 'x1 x25'
 'x1 x26' 'x1 x27' 'x1 x28' 'x1 x29' 'x2^2' 'x2 x3' 'x2 x4' 'x2 x5'
 'x2 x6' 'x2 x7' 'x2 x8' 'x2 x9' 'x2 x10' 'x2 x11' 'x2 x12' 'x2 x13'
 'x2 x14' 'x2 x15' 'x2 x16' 'x2 x17' 'x2 x18' 'x2 x19' 'x2 x20' 'x2 x21'
 'x2 x22' 'x2 x23' 'x2 x24' 'x2 x25' 'x2 x26' 'x2 x27' 'x2 x28' 'x2 x29'
 'x3^2' 'x3 x4' 'x3 x5' 'x3 x6' 'x3 x

In [88]:
price_doc = reg.predict(X2_test)

In [89]:
#Create a DataFrame for the predictions
predictions_df = pd.DataFrame(price_doc, columns=['price_doc'])

#Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'row ID', row_id)

#Save the predictions to a CSV file
predictions_df.to_csv('35MLPRwithPvaluefeatureselction.csv', index=False)